In [ ]:
import torch
import numpy as np
import torch
import torch.nn as nn

# Check if CUDA is available
device=torch.device("cuda" if torch.cuda.is_available else "cpu")

if device=="cuda":
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
class BasicBlock(nn.Module):
    #Basic Block with indentity downsampling 
    def __init__(self,in_channels,out_channels,identity_downsample=None,stride=1):
      super(BasicBlock,self).__init__()
      self.conv1= nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1)
      self.bn1= nn.BatchNorm2d(out_channels)
      self.conv2= nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1)
      self.bn2= nn.BatchNorm2d(out_channels)
      self.relu= nn.ReLU()
      self.identity_downsample= identity_downsample
    
    def forward(self,x):
      identity=x
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu(x)
      x = self.conv2(x)
      x = self.bn2(x)
      x = self.relu(x)
      if self.identity_downsample is not None:
        identity = self.identity_downsample(identity)
      
      # Shortcut connection
      x += identity
      x = self.relu(x)
      return x

In [ ]:
class ResNet32(nn.Module):
    def __init__(self,block,layers,image_channels,num_classes):
      super(ResNet32,self).__init__()
      self.in_channels = 16
      self.conv1 = nn.Conv2d(image_channels,16,kernel_size= 3,stride=1,padding=1)
      self.bn1 = nn.BatchNorm2d(16)
      self.relu = nn.ReLU()
      self.layer1 = self._make_layer(block,5,out_channels=16,stride=1)
      self.layer2 = self._make_layer(block,5,out_channels=32,stride=2)
      self.layer3 = self._make_layer(block,5,out_channels=64,stride=2)
      self.avgpool = nn.AdaptiveAvgPool2d(1)
      self.fc = nn.Linear(64,num_classes)
      self.softmax = nn.Softmax(dim=1)
  
    def forward(self,x): 
      # First Convolutional layer
      x = self.conv1(x)
      x = self.bn1(x)
      x = self.relu(x)
      # Layers with Feature map size = 32
      x = self.layer1(x)
      # Layers with Feature map size = 16
      x = self.layer2(x)
      # Layers wiht Feature map size = 8
      x = self.layer3(x)
      #Average pooling and Output layer
      x = self.avgpool(x)
      x = x.reshape(x.shape[0],-1)
      x = self.fc(x)
      x = self.softmax(x)
      return x

    def _make_layer(self,block,num_residual_blocks,out_channels,stride):
      identity_downsample= None
      layers = []

      #Identity Downsampling
      if stride!=1 or self.in_channels!= out_channels:
        identity_downsample = nn.Sequential(nn.Conv2d(self.in_channels,out_channels,kernel_size=1,stride=stride),
                                            nn.BatchNorm2d(out_channels))
      layers.append(block(self.in_channels,out_channels,identity_downsample,stride))
      
      self.in_channels = out_channels
      for i in range(num_residual_blocks - 1):
          layers.append(block(self.in_channels,out_channels))

      #Stacking all layers
      return nn.Sequential(*layers)


In [ ]:
def ResNet_32(img_channels=3,num_classes = 10):
    return ResNet32(BasicBlock,[5,5,5,5], img_channels,num_classes)

In [ ]:
from torchsummary import summary

#Moving model to gpu if available and Printing model summary
model = ResNet_32(3, 10).to(device)
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           2,320
       BatchNorm2d-5           [-1, 16, 32, 32]              32
              ReLU-6           [-1, 16, 32, 32]               0
            Conv2d-7           [-1, 16, 32, 32]           2,320
       BatchNorm2d-8           [-1, 16, 32, 32]              32
              ReLU-9           [-1, 16, 32, 32]               0
             ReLU-10           [-1, 16, 32, 32]               0
       BasicBlock-11           [-1, 16, 32, 32]               0
           Conv2d-12           [-1, 16, 32, 32]           2,320
      BatchNorm2d-13           [-1, 16, 32, 32]              32
             ReLU-14           [-1, 16,